In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [2]:
merged_dataset = pd.read_csv("C:/Users/Muhammad Mitkar/Desktop/LSN/merged_datasetokis.csv")

In [3]:
print(merged_dataset.info())
print(merged_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32337 entries, 0 to 32336
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      32337 non-null  object
 1   Language  32337 non-null  object
dtypes: object(2)
memory usage: 505.4+ KB
None
                                                Text  Language
0  klement gottwaldi surnukeha palsameeriti ning ...  Estonian
1  sebes joseph pereira thomas  på eng the jesuit...   Swedish
2  ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...      Thai
3  விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...     Tamil
4  de spons behoort tot het geslacht haliclona en...     Dutch


In [4]:
print(merged_dataset.describe(include='all'))

                                                     Text Language
count                                               32337    32337
unique                                              32126       30
top     haec commentatio automatice praeparata res ast...  English
freq                                                   48     2385


In [5]:
merged_dataset.dropna(inplace=True)
merged_dataset.drop_duplicates(inplace=True)

In [6]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text
# Apply text cleaning to the 'Text' column
merged_dataset['Text'] = merged_dataset['Text'].apply(clean_text)


In [7]:
from sklearn.preprocessing import LabelEncoder

# Encode the language labels
label_encoder = LabelEncoder()
merged_dataset['Language_Encoded'] = label_encoder.fit_transform(merged_dataset['Language'])


In [8]:
# Split the dataset into features and labels
texts = merged_dataset['Text'].values
labels = merged_dataset['Language_Encoded'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [9]:
# Initialize and fit the Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
# Pad sequences to ensure uniform length
X_train_padded = pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Define the MLP model
mlp_model = Sequential([
    Flatten(input_shape=(100,)),  # Flatten the input
    Dense(128, activation='relu'),  # First hidden layer
    Dropout(0.3),  # Dropout for regularization
    Dense(64, activation='relu'),  # Second hidden layer
    Dropout(0.3),  # Dropout for regularization
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer (multi-class classification)
])

# Compile the model
mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


C:\Users\Muhammad Mitkar\Desktop\LSN\NEW-VENV\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Train the model
history = mlp_model.fit(
    X_train_padded, y_train,
    epochs=10,  # You can adjust the number of epochs
    validation_data=(X_test_padded, y_test),
    batch_size=32  # Adjust based on your system's memory capacity
)


Epoch 1/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0571 - loss: 177.3631 - val_accuracy: 0.0735 - val_loss: 3.3330
Epoch 2/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0754 - loss: 3.7934 - val_accuracy: 0.0735 - val_loss: 3.3093
Epoch 3/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0741 - loss: 3.3934 - val_accuracy: 0.0735 - val_loss: 3.3035
Epoch 4/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0735 - loss: 3.3375 - val_accuracy: 0.0735 - val_loss: 3.3018
Epoch 5/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0722 - loss: 3.3285 - val_accuracy: 0.0735 - val_loss: 3.3013
Epoch 6/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0720 - loss: 3.3248 - val_accuracy: 0.0735 - val_loss: 3.3011
Epoch 7/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0736 - loss: 3.3138 - val_accuracy: 0.0735 - val_loss: 3.3009
Epoch 8/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0772 - loss: 3.3243 - val_accuracy: 

In [12]:
# Evaluate the model on test data
loss, accuracy = mlp_model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0693 - loss: 3.3023
Test Loss: 3.3008241653442383
Test Accuracy: 0.07345160096883774


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the Bidirectional LSTM model
model = Sequential([
    # Embedding Layer
    Embedding(input_dim=5000,  # Vocabulary size (must match tokenizer num_words)
              output_dim=128,  # Embedding vector size
              input_length=100),  # Max sequence length (must match padding)
    
    # Bidirectional LSTM Layer
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),  # Dropout for regularization
    
    # Additional LSTM layer for deeper learning
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    
    # Fully connected layer
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Output Layer
    Dense(len(label_encoder.classes_), activation='softmax')  # Number of classes in the dataset
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]
)


C:\Users\Muhammad Mitkar\Desktop\LSN\NEW-VENV\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 107s 126ms/step - accuracy: 0.3442 - loss: 2.2565 - val_accuracy: 0.8655 - val_loss: 0.4726
Epoch 2/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 110s 137ms/step - accuracy: 0.8266 - loss: 0.6248 - val_accuracy: 0.9002 - val_loss: 0.3471
Epoch 3/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 106s 132ms/step - accuracy: 0.8948 - loss: 0.3888 - val_accuracy: 0.9102 - val_loss: 0.2992
Epoch 4/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 105s 130ms/step - accuracy: 0.9102 - loss: 0.3131 - val_accuracy: 0.9135 - val_loss: 0.2957
Epoch 5/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 101s 126ms/step - accuracy: 0.9126 - loss: 0.2943 - val_accuracy: 0.9144 - val_loss: 0.3000
Epoch 6/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 103s 128ms/step - accuracy: 0.9195 - loss: 0.2653 - val_accuracy: 0.9167 - val_loss: 0.2876
Epoch 7/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 105s 131ms/step - accuracy: 0.9229 - loss: 0.2505 - val_accuracy: 0.9118 - val_loss: 0.3237
Epoch 8/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 105s 131ms/step - accuracy: 0.9259 -

In [14]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict and evaluate classification performance
y_pred = np.argmax(model.predict(X_test_padded), axis=1)

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


201/201 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9197 - loss: 0.2819
Test Loss: 0.28764083981513977
Test Accuracy: 0.9167444705963135
201/201 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step
              precision    recall  f1-score   support

      Arabic       1.00      0.97      0.98       281
     Chinese       0.40      0.77      0.52       198
      Danish       0.86      0.85      0.86        89
       Dutch       0.99      0.96      0.97       337
     English       0.93      0.96      0.95       472
    Estonian       0.99      0.93      0.96       212
      French       0.96      0.98      0.97       424
      German       0.95      0.92      0.93        98
       Greek       0.98      0.84      0.91        51
       Hindi       1.00      0.99      1.00       213
  Indonesian       0.98      0.98      0.98       200
     Italian       0.92      0.91      0.91       129
    Japanese       0.33      0.14      0.20       187
     Kannada       1.00      0.88      0.93        73
      

In [15]:
def predict_language(input_text, tokenizer, model, label_encoder, max_len=100):
    """
    Predict the language of a given input text.

    Args:
        input_text (str): The text to classify.
        tokenizer (Tokenizer): The fitted tokenizer used for text preprocessing.
        model (Sequential): The trained model.
        label_encoder (LabelEncoder): The fitted label encoder for decoding labels.
        max_len (int): Maximum sequence length for padding (default is 100).

    Returns:
        str: Predicted language.
    """
    # Preprocess the input text
    def clean_text(text):
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\d+', '', text)      # Remove numbers
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        return text

    cleaned_text = clean_text(input_text)
    text_sequence = tokenizer.texts_to_sequences([cleaned_text])
    text_padded = pad_sequences(text_sequence, maxlen=max_len, padding='post', truncating='post')

    # Predict the language
    prediction = model.predict(text_padded)
    predicted_label = np.argmax(prediction, axis=1)[0]
    predicted_language = label_encoder.inverse_transform([predicted_label])[0]

    return predicted_language

# Example usage
input_text = " 彼はずっと世界中を旅してさまざまな文化を体験."  # Replace with your input text
predicted_language = predict_language(input_text, tokenizer, model, label_encoder)
print(f"The predicted language is: {predicted_language}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
The predicted language is: Chinese


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Define the GRU model
gru_model = Sequential([
    # Embedding Layer
    Embedding(input_dim=5000,  # Vocabulary size (must match tokenizer num_words)
              output_dim=128,  # Embedding vector size
              input_length=100),  # Max sequence length (must match padding)
    
    # Bidirectional GRU Layer
    Bidirectional(GRU(128, return_sequences=True)),  # GRU with return_sequences for stacking
    Dropout(0.3),  # Dropout for regularization
    
    # Additional GRU layer
    Bidirectional(GRU(64)),
    Dropout(0.3),
    
    # Fully connected layer
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Output Layer
    Dense(len(label_encoder.classes_), activation='softmax')  # Number of classes
])

# Compile the GRU model
gru_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the GRU model summary
gru_model.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the GRU model
gru_history = gru_model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]
)

# Evaluate the GRU model on the test set
test_loss, test_accuracy = gru_model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict and evaluate classification performance
y_pred = np.argmax(gru_model.predict(X_test_padded), axis=1)

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


C:\Users\Muhammad Mitkar\Desktop\LSN\NEW-VENV\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 99s 117ms/step - accuracy: 0.3383 - loss: 2.2400 - val_accuracy: 0.8953 - val_loss: 0.3763
Epoch 2/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 101s 126ms/step - accuracy: 0.8815 - loss: 0.4407 - val_accuracy: 0.9194 - val_loss: 0.2859
Epoch 3/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 90s 112ms/step - accuracy: 0.9070 - loss: 0.3372 - val_accuracy: 0.9119 - val_loss: 0.2969
Epoch 4/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 92s 115ms/step - accuracy: 0.9168 - loss: 0.2785 - val_accuracy: 0.9135 - val_loss: 0.2968
Epoch 5/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 91s 114ms/step - accuracy: 0.9214 - loss: 0.2646 - val_accuracy: 0.9167 - val_loss: 0.2920
Epoch 6/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 92s 114ms/step - accuracy: 0.9219 - loss: 0.2508 - val_accuracy: 0.9175 - val_loss: 0.2965
Epoch 7/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 93s 116ms/step - accuracy: 0.9271 - loss: 0.2462 - val_accuracy: 0.9153 - val_loss: 0.3111
201/201 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9220 - loss: 0.2828
Test L

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample input text (change this to test different languages)
sample_text = [ "HELLO MY NAME IS MUHAMMAD "]  # Exampช่วยพูดช้าๆได้ไหมle in French

# Step 2: Tokenize the input text
sample_text_sequences = tokenizer.texts_to_sequences(sample_text)

# Step 3: Pad the sequences to match the input length of the model
sample_text_padded = pad_sequences(sample_text_sequences, maxlen=100, padding='post', truncating='post')

# Step 4: Use the GRU model to predict
prediction = gru_model.predict(sample_text_padded)

# Step 5: Decode the predicted label
predicted_label_index = np.argmax(prediction, axis=1)[0]
predicted_language = label_encoder.inverse_transform([predicted_label_index])[0]

print(f"Input text: {sample_text[0]}")
print(f"Predicted language: {predicted_language}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Input text: HELLO MY NAME IS MUHAMMAD 
Predicted language: English


In [18]:
# Save the model to an HDF5 file
gru_model.save('gru_language_detection_model.h5')


In [45]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

# Save the tokenizer to a JSON file
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    json.dump(tokenizer_json, f)

print("Tokenizer saved to 'tokenizer.json'.")


Tokenizer saved to 'tokenizer.json'.


In [43]:
import pickle  # Import the pickle module

# Save the label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("Label encoder saved.")


Label encoder saved.
